In [1]:
import pandas as pd
import os

In [2]:
# SETTINGS
path = '../010-add_coordinates/coordinates/'
filter_df = pd.read_csv('categories.csv', sep = ';', encoding = "ISO-8859-1")
filter_df

,SELECT,Geometry Code,Layer,fclass,Description,OSM Tag,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,accessibility,2601,money,bank,A bank.,amenity=bank,NaN,NaN,NaN,NaN,NaN,NaN
1,accessibility,2602,money,atm,A machine that lets you withdraw amenity=atm,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,accessibility,2901,miscpoi,toilet,Public toilets.,amenity=toilets,NaN,NaN,NaN,NaN,NaN,NaN
3,accessibility,2902,miscpoi,bench,A public bench.,amenity=bench,NaN,NaN,NaN,NaN,NaN,NaN
4,accessibility,2903,miscpoi,drinking_water,A tap or other source of drinking,amenity=drinking_water,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
363,NaN,7223,landuse,plant_nursery,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
364,NaN,7224,landuse,brownfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
365,NaN,7225,landuse,greenfield,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
366,NaN,7226,landuse,construction,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
filter_df = filter_df.loc[~filter_df.SELECT.isna(), ['SELECT', 'Geometry Code', 'Layer', 'fclass']]
#filter_df['Geometry Code'] = filter_df['Geometry Code'].str.pad(4, side = 'right', fillchar = '.')
filter_df['Geometry Code'] = filter_df['Geometry Code'].str.strip('0')

In [4]:
granularity = 4 - filter_df['Geometry Code'].str.len()
multiplier = 10 ** granularity

In [5]:
filter_df['Geometry Code'] = filter_df['Geometry Code'].astype(int)

filter_df['low_threshold'] = multiplier * filter_df['Geometry Code']
filter_df['high_threshold'] = multiplier * filter_df['Geometry Code'] + (multiplier - 1)

filter_df['fclass'] = filter_df['fclass'].fillna(filter_df['Layer'])
filter_df['granularity'] = granularity

filter_df = filter_df.sort_values(by=['granularity'])

filter_df

,SELECT,Geometry Code,Layer,fclass,low_threshold,high_threshold,granularity
0,accessibility,2601,money,bank,2601,2601,0
139,sport,2251,leisure,sports_centre,2251,2251,0
140,sport,2252,leisure,pitch,2252,2252,0
141,sport,2253,leisure,swimming_pool,2253,2253,0
142,sport,2254,leisure,tennis_court,2254,2254,0
...,...,...,...,...,...,...,...
242,transports,52,traffic,traffic,5200,5299,2
122,nature,82,water,water,8200,8299,2
243,transports,61,railway,railway,6100,6199,2
240,transports,5,transport,transport,5000,5999,3


In [6]:
file_list = os.listdir(path)

filter_regex = '|'.join(r"{}".format(x) for x in filter_df['Geometry Code'])

In [7]:
filter_regex

'2601|2251|2252|2253|2254|2255|2256|2257|2522|2524|2524|5209|2542|2401|2402|2403|2404|2405|2406|2421|2422|2423|2424|2905|5208|2923|2923|7228|5656|8101|8102|8103|8104|6105|8202|8211|8221|6104|6103|6102|8201|2001|2002|2011|2101|2111|2129|2907|2921|2922|2701|2704|2705|2706|5144|5145|5146|5147|5151|5152|5153|5154|5155|5199|5251|5261|5262|5263|5601|5602|5603|5621|5622|5641|5651|5652|5655|5143|7219|5142|5135|2721|6101|5671|1001|1002|1003|2562|2563|2565|5111|5112|5113|5114|5115|5121|5122|5123|5124|5125|5131|5132|5133|5134|5141|7218|7229|7216|3109|7227|6119|3301|3302|6115|6114|6113|6112|6111|6109|2007|2016|2202|2203|2205|2206|2301|2302|2303|2304|2305|7217|3108|3107|3106|3105|2602|2901|2902|2903|2904|5204|2014|2015|2201|2722|2723|2307|2724|2731|2732|2734|2735|2736|2737|2951|3101|3102|3103|3104|2725|2501|2306|2503|2034|2204|2502|2547|2733|2741|2742|2743|2906|4101|2033|4103|4112|4113|4121|4132|4141|5311|5321|7201|7202|7206|4111|2032|5661|2744|2515|2511|2518|2505|2031|2521|2529|2529|2512|2954|2513

In [8]:
df_dict = {}

for file_name in file_list:

    df_dict[file_name] = pd.read_csv(path + file_name)

    df_dict[file_name].drop('Unnamed: 0', axis = 1, inplace = True)

In [9]:
filter_df_to_dict = filter_df[['fclass', 'low_threshold', 'high_threshold']]
filter_df_to_dict.drop_duplicates(subset = ['fclass'], inplace = True)
filter_df_to_dict

C:\Users\gabri\AppData\Local\Temp\ipykernel_104076\544379905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df_to_dict.drop_duplicates(subset = ['fclass'], inplace = True)


,fclass,low_threshold,high_threshold
0,bank,2601,2601
139,sports_centre,2251,2251
140,pitch,2252,2252
141,swimming_pool,2253,2253
142,tennis_court,2254,2254
...,...,...,...
116,water,8200,8299
120,natural,4100,4199
242,traffic,5200,5299
240,transport,5000,5999


In [10]:
filter_df_to_dict.index = filter_df_to_dict.fclass
filter_df_to_dict.drop('fclass', axis = 1, inplace = True)
filter_dict = filter_df_to_dict.to_dict('index')

C:\Users\gabri\AppData\Local\Temp\ipykernel_104076\3101066230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter_df_to_dict.drop('fclass', axis = 1, inplace = True)


In [11]:
filter_df

,SELECT,Geometry Code,Layer,fclass,low_threshold,high_threshold,granularity
0,accessibility,2601,money,bank,2601,2601,0
139,sport,2251,leisure,sports_centre,2251,2251,0
140,sport,2252,leisure,pitch,2252,2252,0
141,sport,2253,leisure,swimming_pool,2253,2253,0
142,sport,2254,leisure,tennis_court,2254,2254,0
...,...,...,...,...,...,...,...
242,transports,52,traffic,traffic,5200,5299,2
122,nature,82,water,water,8200,8299,2
243,transports,61,railway,railway,6100,6199,2
240,transports,5,transport,transport,5000,5999,3


In [12]:
# df = df_dict['coordinates_gis_osm_pois_free_1.csv']
#
# df['fclass'] = 0
#
# for fclass, value_dict in filter_dict.items():
#
#     df.loc[(df.code >= value_dict['low_threshold']) & (df.code <= value_dict['high_threshold']),'fclass'] = fclass
#
#     final_df = df.merge(filter_df[['fclass', 'SELECT']], on = 'fclass', how = 'left')
#
#     final_df = final_df[final_df.fclass != 0]
#
# final_df

In [13]:
filtered_df_dict = {}

for df_name, df in df_dict.items():

    print(df_name)

    df['fclass'] = 0

    for fclass, value_dict in filter_dict.items():

        df.loc[(df.code >= value_dict['low_threshold']) & (df.code <= value_dict['high_threshold']),'fclass'] = fclass

        final_df = df.merge(filter_df[['fclass', 'SELECT']], on = 'fclass', how = 'left')

        final_df = final_df.drop_duplicates()

        final_df = final_df[final_df.fclass != 0]

    filtered_df_dict[df_name] = final_df

In [14]:
for df_name in filtered_df_dict.keys():
    print('DATAFRAME NAME: ', df_name)
    print('ORIGINAL DATAFRAME SIZE: ', df_dict[df_name].shape)
    print('FILTERED DATAFRAME SIZE: ', filtered_df_dict[df_name].shape)
    print('DIFFERENCE: ', df_dict[df_name].shape[0] - filtered_df_dict[df_name].shape[0])
    print(' ')

DATAFRAME NAME:  coordinates_flevoland_gis_osm_buildings_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (252302, 7)
FILTERED DATAFRAME SIZE:  (0, 8)
DIFFERENCE:  252302
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_landuse_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (47999, 6)
FILTERED DATAFRAME SIZE:  (46265, 7)
DIFFERENCE:  1734
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_natural_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (100, 6)
FILTERED DATAFRAME SIZE:  (100, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_natural_free_1.csv
ORIGINAL DATAFRAME SIZE:  (8365, 6)
FILTERED DATAFRAME SIZE:  (8365, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_places_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (59, 7)
FILTERED DATAFRAME SIZE:  (1, 8)
DIFFERENCE:  58
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_places_free_1.csv
ORIGINAL DATAFRAME SIZE:  (57, 7)
FILTERED DATAFRAME SIZE:  (18, 8)
DIFFERENCE:  39
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_pofw_a_free_1.c

In [15]:
completed_df_dict = {}

for df_name, df in filtered_df_dict.items():

    if df.shape[0] > 0:
        # completed_df_dict[df_name] = df[~df.name.isna()]
        completed_df_dict[df_name] = df

In [16]:
for df_name in completed_df_dict.keys():
    print('DATAFRAME NAME: ', df_name)
    print('ORIGINAL DATAFRAME SIZE: ', df_dict[df_name].shape)
    print('FILTERED DATAFRAME SIZE: ', filtered_df_dict[df_name].shape)
    print('COMPLETE DATAFRAME SIZE: ', completed_df_dict[df_name].shape)
    print('DIFFERENCE: ', df_dict[df_name].shape[0] - completed_df_dict[df_name].shape[0])
    print(' ')

DATAFRAME NAME:  coordinates_flevoland_gis_osm_landuse_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (47999, 6)
FILTERED DATAFRAME SIZE:  (46265, 7)
COMPLETE DATAFRAME SIZE:  (46265, 7)
DIFFERENCE:  1734
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_natural_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (100, 6)
FILTERED DATAFRAME SIZE:  (100, 7)
COMPLETE DATAFRAME SIZE:  (100, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_natural_free_1.csv
ORIGINAL DATAFRAME SIZE:  (8365, 6)
FILTERED DATAFRAME SIZE:  (8365, 7)
COMPLETE DATAFRAME SIZE:  (8365, 7)
DIFFERENCE:  0
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_places_a_free_1.csv
ORIGINAL DATAFRAME SIZE:  (59, 7)
FILTERED DATAFRAME SIZE:  (1, 8)
COMPLETE DATAFRAME SIZE:  (1, 8)
DIFFERENCE:  58
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_places_free_1.csv
ORIGINAL DATAFRAME SIZE:  (57, 7)
FILTERED DATAFRAME SIZE:  (18, 8)
COMPLETE DATAFRAME SIZE:  (18, 8)
DIFFERENCE:  39
 
DATAFRAME NAME:  coordinates_flevoland_gis_osm_pof

In [17]:
os.makedirs('filtered/', exist_ok=True)

for df_name, df in completed_df_dict.items():

    file_name = 'filtered_' + df_name[:-4] + '.csv'
    df.to_csv('filtered/' + file_name)

In [18]:
completed_df_dict['coordinates_gis_osm_water_a_free_1.csv']

KeyError: 'coordinates_gis_osm_water_a_free_1.csv'